In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
df=pd.read_csv("eco2mix_Data.csv",index_col="Unnamed: 0")
df_train=df[df['Date - Heure']<"2025-10-01"]
df_test=df[df['Date - Heure']>"2025-10-01"]


In [ ]:
from pathlib import Path
from typing import List, Union

import numpy as np
from gluonts.dataset.arrow import ArrowWriter


def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    compression: str = "lz4",
):
    """
    Store a given set of series into Arrow format at the specified path.

    Input data can be either a list of 1D numpy arrays, or a single 2D
    numpy array of shape (num_series, time_length).
    """
    assert isinstance(time_series, list) or (
        isinstance(time_series, np.ndarray) and
        time_series.ndim == 2
    )

    # Set an arbitrary start time
    start = np.datetime64("2000-01-01 00:00", "s")

    dataset = [
        {"start": start, "target": ts} for ts in time_series
    ]

    ArrowWriter(compression=compression).write_to_file(
        dataset,
        path=path,
    )


if __name__ == "__main__":
    # Generate 20 random time series of length 1024
    time_series = [df_train["target"].values]

    # Convert to GluonTS arrow format
    convert_to_arrow("./data.arrow", time_series=time_series)

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python /content/chronos-forecasting/scripts/training/train.py --config /content/chronos-forecasting/scripts/training/configs/chronos-t5-small.yaml \
    --model-id amazon/chronos-t5-small \
    --no-random-init \
    --max-steps 1000 \
    --learning-rate 0.001

2025-11-22 22:56:09.079926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763852169.101261   19503 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763852169.107462   19503 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763852169.123211   19503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763852169.123236   19503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763852169.123240   19503 computation_placer.cc:177] computation placer alr

In [ ]:
#!pip install "chronos-forecasting[dev] @ git+https://github.com/amazon-science/chronos-forecasting.git"

In [ ]:
#!git clone https://github.com/amazon-science/chronos-forecasting.git

In [ ]:
#!pip install triton